In [ ]:
%matplotlib inline
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import numpy as np
import keras
from keras.datasets import cifar10
from keras.layers import Input, Dense, Conv2D, AveragePooling2D, Activation, GlobalAveragePooling2D, Lambda
from keras.layers.core import Dropout
from keras.layers.merge import Concatenate
from keras.layers.normalization import BatchNormalization
from keras.initializers import Initializer
from keras.regularizers import l2
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler, EarlyStopping, ModelCheckpoint

from PIL import Image
import matplotlib.pyplot as plt

## Prepare dataset and dataset augmentation

In [ ]:
(xx_train, yy_train), (x_test, y_test) = cifar10.load_data()
x_train = xx_train[:40000]
y_train = yy_train[:40000]

x_validate = xx_train[40000:50000]
y_validate = yy_train[40000:50000]

x_train = x_train/255.0
x_validate = x_validate/255.0
x_test = x_test/255.0

y_train = to_categorical(y_train, 10)
y_validate = to_categorical(y_validate, 10)
y_test = to_categorical(y_test, 10)

In [ ]:
'''TRAINING DATA GENERATOR FOR DATA AUGMENTATION'''
train_datagen = ImageDataGenerator(
    rotation_range = 40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode = 'nearest'
)

train_datagen.fit(x_train)

## Define model

In [ ]:
def dense_block(x,num_layers_per_block,growth_rate):
    for i in range(num_layers_per_block//2):
        x_ = BatchNormalization()(x)
        x_ = Activation('relu')(x_)
        x_ = Conv2D(number_filters,(3,3),padding='same',kernel_initializer='he_normal',kernel_regularizer=l2(0.0001))(x_)
        x_ = Dropout(0.2)(x_)
        x_ = BatchNormalization()(x_)
        x_ = Activation('relu')(x_)
        x = Concatenate()([x,x_])
    return x 

def transition_layers(x,compression):
    updated_num_filters = int(x.get_shape().as_list()[-1] * compression)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(updated_num_filters,(1,1),padding='same',kernel_initializer='he_normal',kernel_regularizer=l2(0.0001))(x)
    x = Dropout(0.2)(x)
    x = AveragePooling2D()(x)
    return x

In [ ]:
#Define model parameters
model_depth = 100
num_dense_blocks = 3
growth_rate = 12
number_filters = 16
compression = 0.5
num_layers_per_block = (model_depth - 4) // num_dense_blocks

In [ ]:
#Let's define the model
inp = Input(x_train.shape[1:])
x = Conv2D(number_filters,(1,1),padding='same',kernel_initializer='he_normal',kernel_regularizer=l2(0.0001))(inp)
for i in range(num_dense_blocks):
    x = dense_block(x,num_layers_per_block,growth_rate)
    if (i != num_dense_blocks-1):
        x = transition_layers(x,compression)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = GlobalAveragePooling2D()(x)
x = Dense(10, activation='softmax',kernel_initializer='he_normal',kernel_regularizer=l2(0.0001))(x)

model = Model(inp, x)
model.compile(Adam(), loss = 'categorical_crossentropy',metrics=['accuracy'])

## Define callbacks

In [ ]:
'''Callbacks'''

training_losses = []
validation_losses = []

class LossHistory(keras.callbacks.Callback):
    def on_epoch_end(self,epoch, logs = {}):
        training_losses.append(logs.get('loss'))
        validation_losses.append(logs.get('val_loss'))
        
model_loss = LossHistory()

checkpoint = ModelCheckpoint('model.h5', 
                             monitor = 'val_acc', 
                             verbose = 1,
                             save_best_only = True,
                             save_weights_only = False, 
                             mode = 'auto', 
                             period = 1
                            )

early_stop = EarlyStopping(monitor = 'val_acc', 
                           min_delta = 0, 
                           patience = 40, 
                           verbose = 0,
                           mode = 'auto'
                          )

## Train the model

In [ ]:
epochs = 150
init_lr = 0.0001
dr = init_lr/epochs

lr = init_lr - (len(training_losses) * dr)
model.optimizer = Adam(lr, decay = dr)

'''model.fit(x_train, y_train, 
          batch_size = 32, 
          epochs = 150, 
          shuffle = True,
          validation_data = (x_validate, y_validate),
          callbacks=[earlyStopping]
         )'''

model.fit_generator(train_datagen.flow(x_train, y_train, batch_size = 128),
                    validation_data = (x_validate, y_validate),
                    steps_per_epoch = len(x_train) / 32, 
                    epochs = epochs,
                    callbacks = [checkpoint, early_stop, model_loss]
                    )

## Evaluate model

In [ ]:
'''Plot training and validation losses'''

plt.plot(training_losses, label='Training Loss')
plt.plot(validation_losses, label='Validation Loss')
plt.legend(loc = 'upper right')

In [ ]:
'''Load best model and evaluate on test set'''

best_model = load_model('model.h5')
best_model.evaluate(x_test, y_test)

evaluation = best_model.evaluate(x_test, y_test, 
                            batch_size = 512, 
                            verbose = 1
                           )

print('Loss: %.2f Accuracy: %.2f%%' % (evaluation[0], evaluation[1]*100.0))